# Simple Metadata Example


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


## Example description

In this example we will define a following metadata
```
name: my-model-name
versions: [ my-model-version-01 ]
platform: seldon
inputs:
- messagetype: tensor
  schema:
    names: [a, b, c, d]
    shape: [ 4 ]
outputs:
- messagetype: tensor
  schema:
    shape: [ 1 ]
```

which corresponds to model taking a tensor input with four columns named `a`, `b`, `c`, and `d`.

We will define the `metadata` in two ways: directly in the model.py and in the depployment manifest.

In [3]:
import requests
import time
def getWithRetry(url):
    for i in range(3):
        r = requests.get(url)
        if r.status_code == requests.codes.ok:
            meta = r.json()
            return meta
        else:
            print("Failed request with status code ",r.status_code)
            time.sleep(3)

## 1. Directly define in model

In [4]:
%%writefile models/init-metadata/Model.py

import logging


class Model:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features

    def init_metadata(self):
        logging.info("metadata method  called")

        meta = {
            "name": "my-model-name",
            "versions": ["my-model-version-01"],
            "platform": "seldon",
            "inputs": [
                {
                    "messagetype": "tensor",
                    "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
                }
            ],
            "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
        }

        return meta

Overwriting models/init-metadata/Model.py


In [5]:
%%writefile model-metadata/init-metadata.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model-init-metadata
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metadata:0.3
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/init-metadata.yaml


In [6]:
!kubectl apply -f model-metadata/init-metadata.yaml

seldondeployment.machinelearning.seldon.io/seldon-model-init-metadata configured


In [7]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model-init-metadata -o jsonpath='{.items[0].metadata.name}')

deployment "seldon-model-init-metadata-example-0-my-model" successfully rolled out


### Model Metadata

In [8]:
meta = getWithRetry("http://localhost:8003/seldon/seldon/seldon-model-init-metadata/api/v1.0/metadata/my-model")

assert meta == {
    "name": "my-model-name",
    "versions": ["my-model-version-01"],
    "platform": "seldon",
    "inputs": [{
        "messagetype": "tensor",
        "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
    }],
    "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
}

meta

{'inputs': [{'messagetype': 'tensor',
   'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
 'name': 'my-model-name',
 'outputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}],
 'platform': 'seldon',
 'versions': ['my-model-version-01']}

### Graph Metadata

In [9]:
meta = getWithRetry("http://localhost:8003/seldon/seldon/seldon-model-init-metadata/api/v1.0/metadata")

assert meta == {
    "name": "example",
    "models": {
        "my-model": {
            "name": "my-model-name",
            "platform": "seldon",
            "versions": ["my-model-version-01"],
            "inputs": [{
                "messagetype": "tensor",
                "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
            }],
            "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
        }
    },
    "graphinputs": [{
        "messagetype": "tensor",
        "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
    }],
    "graphoutputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
}

meta

{'name': 'example',
 'models': {'my-model': {'name': 'my-model-name',
   'platform': 'seldon',
   'versions': ['my-model-version-01'],
   'inputs': [{'messagetype': 'tensor',
     'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
   'outputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}]}},
 'graphinputs': [{'messagetype': 'tensor',
   'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
 'graphoutputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}]}

## 2. Via environmental variable

In [10]:
%%writefile model-metadata/environ-metadata.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model-environ-metadata
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node_rest:0.3
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              ---
              name: my-model-name
              versions: [ my-model-version-01 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [a, b, c, d]
                  shape: [4]
              outputs:
              - messagetype: tensor
                schema:
                  shape: [ 1 ]
    graph:
      children: []
      name: my-model
      type: MODEL
    name: example
    replicas: 1

Overwriting model-metadata/environ-metadata.yaml


In [11]:
!kubectl apply -f model-metadata/environ-metadata.yaml

seldondeployment.machinelearning.seldon.io/seldon-model-environ-metadata configured


In [12]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model-environ-metadata -o jsonpath='{.items[0].metadata.name}')

deployment "seldon-model-environ-metadata-example-0-my-model" successfully rolled out


### Model Metadata

In [13]:
meta = getWithRetry("http://localhost:8003/seldon/seldon/seldon-model-environ-metadata/api/v1.0/metadata/my-model")


assert meta == {
    "name": "my-model-name",
    "versions": ["my-model-version-01"],
    "platform": "seldon",
    "inputs": [{
        "messagetype": "tensor",
        "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
    }],
    "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
}

meta

{'inputs': [{'messagetype': 'tensor',
   'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
 'name': 'my-model-name',
 'outputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}],
 'platform': 'seldon',
 'versions': ['my-model-version-01']}

### Graph Metadata

In [14]:
meta = getWithRetry("http://localhost:8003/seldon/seldon/seldon-model-init-metadata/api/v1.0/metadata")

assert meta == {
    "name": "example",
    "models": {
        "my-model": {
            "name": "my-model-name",
            "platform": "seldon",
            "versions": ["my-model-version-01"],
            "inputs": [{
                "messagetype": "tensor",
                "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
            }],
            "outputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
        }
    },
    "graphinputs": [{
        "messagetype": "tensor",
        "schema": {"names": ["a", "b", "c", "d"], "shape": [4]},
    }],
    "graphoutputs": [{"messagetype": "tensor", "schema": {"shape": [1]}}],
}

meta

{'name': 'example',
 'models': {'my-model': {'name': 'my-model-name',
   'platform': 'seldon',
   'versions': ['my-model-version-01'],
   'inputs': [{'messagetype': 'tensor',
     'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
   'outputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}]}},
 'graphinputs': [{'messagetype': 'tensor',
   'schema': {'names': ['a', 'b', 'c', 'd'], 'shape': [4]}}],
 'graphoutputs': [{'messagetype': 'tensor', 'schema': {'shape': [1]}}]}

## Cleanup resources

In [15]:
%%bash
kubectl delete -f model-metadata/

seldondeployment.machinelearning.seldon.io "seldon-model-environ-metadata" deleted
seldondeployment.machinelearning.seldon.io "seldon-model-init-metadata" deleted
